In [5]:
import pandas as pd
import numpy as np

from my_functions_idealisedprofiles_drierConditions import *
model_directory = '../../../../FloodModelling/Model_IdealisedProfiles_drierConditions/'

### Get version of landcover array with just 'urban' and 'rural' categories

In [6]:
# Read in the data
landcover, out_meta = prepare_rainfall_scenario_raster("../../../../FloodModelling/LandCoverData/LandCover_clipped.tif", True)
# Convert the 1 and 6 values to 10 (for urban) and the rest to 11 (for non-urban).  
landcover_mod =  np.where(landcover==1, 10, landcover)
landcover_mod =  np.where(landcover_mod==6, 10, landcover_mod)
# Convert the rest of the classes to 11
for i in [1,2,3,4,5,7,8,9]:
    landcover_mod =  np.where(landcover_mod==i, 11, landcover_mod)

### Define the names of the method (shorter and longer versions)

In [7]:
methods =[ '6h_sp_c_0.5','6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4'
         , '6h_sp_bl_0.6', '6h_sp_bl_0.7', '6h_sp_bl_0.8', '6h_sp_bl_0.9']

### Find maximum intensity for each method and minute in which it occurs (to use in sorting results analysis)

In [8]:
maxs = []
min_of_maxs = []

# Add FEH data
# feh_precip=pd.read_csv("../../CreateSyntheticRainfallEvents/ReFH2_singlepeak/6hr_100yrRP/PostLossRemoval/6h_feh_singlepeak_urban.csv")
# maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
# min_of_maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

for method in methods:
    method = method.replace('_scaled', '')
    precip=pd.read_csv("../../CreateSyntheticRainfallEvents/IdealisedProfiles_Scaled/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(method))
    maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
    min_of_maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

### Create versions of lists of methods, in order based on max intensity and the the timing of the max intensity 

In [9]:
short_ids_by_loading=  pd.DataFrame({"min": min_of_maxs, 'method_name': methods}).sort_values('min')["method_name"].tolist()
short_ids_by_intensity = pd.DataFrame({"min": maxs, 'method_name': methods}).sort_values('min', ascending = False)["method_name"].tolist()

### Create dataframe of colours for each cluster (based on their loading)

In [10]:
colours_df= create_colours_df(short_ids_by_loading, methods)

### Create list of filepaths, formatted to be used for either depth or velocity

In [11]:
fps = []
for method_num, short_id in enumerate(methods):
    fp = model_directory + "{}/{} (Max).Resampled.Terrain.tif".format(short_id, '{}')
    fps.append(fp)

### Define breaks for categorising velocity and depth

In [12]:
# Define breaks to split the depths/velocities on
breaks_depths = np.array([0, 0.3, 0.6, 1.2, 100])  
labels_depth = ['<=0.3m', '0.3-0.6m', '0.6-1.2m', '>1.2m']
breaks_velocity = np.array([0,0.25,0.5,2,100])
labels_velocity = ["<=0.25m/s", "0.25-0.5m/s", "0.5-2m/s", ">2m/s"]

# <u> Flood extent </u>
To examine whether the rainfall's temporal distribution influences the total extent of flooding, the number of flooded cells and the total flooded area in km2 (incl. only cells with depth >0.1m) is compared between the profile with a single peak, and the three methods for producing multi-peaked rainfall events. b

### Create dataframes containing the (total/urban) flooded area in each depth/velocity bin

In [13]:
velocity_counts, velocity_props = create_binned_counts_and_props(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values)
depth_counts, depth_props = create_binned_counts_and_props(fps, 'Depth', breaks_depths, labels_depth, remove_little_values)

In [14]:
velocity_counts_urban, velocity_props_urban = create_binned_counts_and_props_urban(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values, landcover_mod)
depth_counts_urban, depth_props_urban = create_binned_counts_and_props_urban(fps, 'Depth', breaks_depths, labels_depth, remove_little_values, landcover_mod)

### Create dataframes containing the (total/urban) flooded area

In [15]:
totals_df = create_totals_df(velocity_counts)
totals_df_urban = create_totals_df(velocity_counts_urban)     

In [16]:
totals_df

,short_id,FloodedArea
0,6h_sp_c_0.5,1.534739
1,6h_sp_fl_0.1,1.465218
2,6h_sp_fl_0.2,1.479458
3,6h_sp_fl_0.3,1.498196
4,6h_sp_fl_0.4,1.517187
5,6h_sp_bl_0.6,1.557027
6,6h_sp_bl_0.7,1.572332
7,6h_sp_bl_0.8,1.585933
8,6h_sp_bl_0.9,1.597071


### Create dataframes containing the % diff in the flooded area between single peak and each other method  

In [17]:
percent_diffs_df = find_percentage_diff (totals_df, fps, baseline_short_id = '6h_sp_c_0.5') 
percent_diffs_df_urban = find_percentage_diff (totals_df_urban, fps, baseline_short_id = '6h_sp_c_0.5')

## Find number of cells in which each method leads to the worst flooding (depth/velocity)

In [18]:
# Find the number of flooded cells with the worst flooding for each method
worst_case_method_depth = find_worst_case_method(fps, methods, 'Depth')
worst_case_method_velocity = find_worst_case_method(fps, methods,  'Velocity') 

In [19]:
# Remove multiple matches and nan
worst_case_method_depth = worst_case_method_depth[~worst_case_method_depth['values'].isin(['multiple matches','nan'])]
worst_case_method_velocity = worst_case_method_velocity[~worst_case_method_velocity['values'].isin(['multiple matches','nan'])]

# # Reorder (and also add in the methods that are missing)
worst_case_method_depth = pd.merge(worst_case_method_depth,  pd.DataFrame({'values': methods}), how="outer")
worst_case_method_depth = worst_case_method_depth.reindex(worst_case_method_depth['values'].map(dict(zip(methods, range(len(methods))))).sort_values().index)
worst_case_method_depth.reset_index(inplace=True,drop=True)

worst_case_method_velocity = pd.merge(worst_case_method_velocity,  pd.DataFrame({'values': methods}), how="outer")
worst_case_method_velocity = worst_case_method_velocity.reindex(worst_case_method_velocity['values'].map(dict(zip(methods, range(len(methods))))).sort_values().index)
worst_case_method_velocity.reset_index(inplace=True,drop=True)

## Find number of cells with each hazard rating

In [20]:
hazard_counts, hazard_props = create_binned_counts_and_props_hazard(fps)

## Find number of cells which have moved between hazard categories

In [21]:
hazard_cat_changes = create_binned_counts_and_props_hazard_cat_change(fps)

### Create a dataframe containing all the info on each of the scenarios

In [22]:
cluster_results = pd.DataFrame({'Cluster_num': methods, "MaxRainfallIntensity": maxs,  
    "MaxRainfallIntensityMinute": min_of_maxs,
   'TotalFloodedArea':totals_df['FloodedArea'],'%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs'],
    '%Diff_FloodedArea_fromSP_formatted':percent_diffs_df['percent_diff_formatted'],
    'Abs%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs_abs'],'UrbanFloodedArea':totals_df_urban['FloodedArea'],
  '%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs'] ,
  '%Diff_UrbanFloodedArea_fromSP_formatted':percent_diffs_df_urban['percent_diff_formatted'],
    'Abs%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs_abs'],'colour':colours_df['colour'],
    'WorstCaseDepth_ncells': worst_case_method_depth['counts'].tolist(),
    'WorstCaseVelocity_ncells': worst_case_method_velocity['counts'].tolist(), }) 

### Add the depth/velocity category breakdowns and hazard categories to this

In [23]:
dfs = [velocity_props, depth_props, velocity_props_urban, depth_props_urban, velocity_counts, depth_counts,
          velocity_counts_urban, depth_counts_urban,hazard_counts, hazard_props]
suffixes = ['_propcells', '_propcells','_propcells_urban','_propcells_urban','_countcells','_countcells','_countcells_urban','_countcells_urban',
'_numcells', '_propcells']

for num, df in enumerate(dfs):
    # Reformat the dataframe
    df = df.set_index('index').T
    # Add the correct suffix to the column names
    df = df.add_suffix(suffixes[num]) 
    # Add Cluster_num column for joining
    df['Cluster_num'] = df.index#
    # Join to cluster results dataframe
    cluster_results = pd.merge(cluster_results,  df, how="outer", on = 'Cluster_num')
    
cluster_results = pd.merge(cluster_results, hazard_cat_changes,  how="outer", on = 'Cluster_num')    

### Save to file

In [26]:
cluster_results.to_csv("Data/allclusters_summary.csv", index=False)
cluster_results

,Cluster_num,MaxRainfallIntensity,MaxRainfallIntensityMinute,TotalFloodedArea,%Diff_FloodedArea_fromSP,%Diff_FloodedArea_fromSP_formatted,Abs%Diff_FloodedArea_fromSP,UrbanFloodedArea,%Diff_UrbanFloodedArea_fromSP,%Diff_UrbanFloodedArea_fromSP_formatted,...,Hazard_SameCat_countcells,Hazard_1CatsHigher_countcells,Hazard_2CatsHigher_countcells,Hazard_3CatsHigher_countcells,Hazard_2CatsLower_propcells,Hazard_1CatsLower_propcells,Hazard_SameCat_propcells,Hazard_1CatsHigher_propcells,Hazard_2CatsHigher_propcells,Hazard_3CatsHigher_propcells
0,6h_sp_c_0.5,0.301343,180,1.534739,0.00,,0.00,0.496018,0.00,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6h_sp_fl_0.1,0.301343,36,1.465218,-4.53,-4.53%,4.53,0.464385,-6.38,-6.38%,...,1365769.0,67118.0,2768.0,2.0,0.0,1.2,94.0,4.6,0.2,0.0
2,6h_sp_fl_0.2,0.301343,72,1.479458,-3.60,-3.6%,3.60,0.471585,-4.93,-4.93%,...,1399856.0,54034.0,1771.0,2.0,0.0,1.0,95.2,3.7,0.1,0.0
3,6h_sp_fl_0.3,0.301343,108,1.498196,-2.38,-2.38%,2.38,0.479371,-3.36,-3.36%,...,1441060.0,38344.0,889.0,2.0,0.0,0.7,96.6,2.6,0.1,0.0
4,6h_sp_fl_0.4,0.301343,144,1.517187,-1.14,-1.14%,1.14,0.487419,-1.73,-1.73%,...,1486938.0,17743.0,277.0,2.0,0.0,0.5,98.3,1.2,0.0,0.0
5,6h_sp_bl_0.6,0.301343,216,1.557027,1.45,+1.45%,1.45,0.504568,1.72,+1.72%,...,1509866.0,4784.0,5.0,NaN,0.0,1.2,98.5,0.3,0.0,NaN
6,6h_sp_bl_0.7,0.301343,252,1.572332,2.45,+2.45%,2.45,0.512602,3.34,+3.34%,...,1475958.0,13289.0,9.0,NaN,0.1,2.4,96.6,0.9,0.0,NaN
7,6h_sp_bl_0.8,0.301343,288,1.585933,3.34,+3.34%,3.34,0.519749,4.78,+4.78%,...,1452944.0,17445.0,21.0,NaN,0.1,3.3,95.5,1.1,0.0,NaN
8,6h_sp_bl_0.9,0.301343,324,1.597071,4.06,+4.06%,4.06,0.525408,5.93,+5.93%,...,1426843.0,28502.0,35.0,NaN,0.2,3.9,94.0,1.9,0.0,NaN


### Delete tiff files (as these aren't used again and take up a lot of space)

In [25]:
# for method in short_ids:
#     print(method)
#     if method != '6h_feh_sp':
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_cat_difference.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_classified.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_posneg.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_classified.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_posneg.tif".format(method)) 
        
#     os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_classified.tif".format(method)) 
#     os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_classified.tif".format(method)) 
#     os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_classified.tif".format(method)) 